In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from stable_baselines3.common.policies import ActorCriticPolicy
from gym.spaces import Box
import gym
from stable_baselines3 import PPO



In [2]:
class Config:
    USE_BATCH_NORM = 1
    DROPOUT = 0
    ARCHITECTURE = 'impala'

In [3]:

class ImpalaBlock(nn.Module):
    def __init__(self, in_channels, use_batch_norm=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.use_batch_norm = use_batch_norm
        
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(in_channels)
            self.bn2 = nn.BatchNorm2d(in_channels)
            
        self.dropout = nn.Dropout2d(p=Config.DROPOUT) if Config.DROPOUT > 0 else None

    def forward(self, x):
        out = F.relu(x)
        
        out = self.conv1(out)
        if self.dropout:
            out = self.dropout(out)
        if self.use_batch_norm:
            out = self.bn1(out)
            
        out = F.relu(out)
        
        out = self.conv2(out)
        if self.dropout:
            out = self.dropout(out)
        if self.use_batch_norm:
            out = self.bn2(out)
            
        return out + x

In [4]:

class ImpalaCNN(nn.Module):
    def __init__(self, num_channels=3, depths=[16, 32, 32]):
        super().__init__()
        self.use_batch_norm = Config.USE_BATCH_NORM == 1
        
        layers = []
        in_channels = num_channels
        
        for depth in depths:
            layers.extend([
                nn.Conv2d(in_channels, depth, 3, padding=1),
                nn.MaxPool2d(3, stride=2, padding=1),
                ImpalaBlock(depth, self.use_batch_norm),
                ImpalaBlock(depth, self.use_batch_norm)
            ])
            in_channels = depth
            
        self.conv_layers = nn.Sequential(*layers)
        self.fc = nn.Linear(self._get_conv_output_size(num_channels), 256)
        
    def _get_conv_output_size(self, channels):
        x = torch.zeros(1, channels, 64, 64)
        x = self.conv_layers(x)
        return int(np.prod(x.shape[1:]))
        
    def forward(self, images):
        x = images.float() / 255.0
        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(x)
        x = F.relu(self.fc(x))
        return x

In [21]:

class CnnPolicy(ActorCriticPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, *args, **kwargs):
        
        super().__init__(
            observation_space,
            action_space,
            lr_schedule,
            *args,
            **kwargs
        )
        
        self.cnn = ImpalaCNN(num_channels=observation_space.shape[0])
        self.actor = nn.Linear(256, action_space.n)
        self.critic = nn.Linear(256, 1)

    def forward(self, obs, deterministic=False):
        features = self.cnn(obs)
        action_logits = self.actor(features)
        value = self.critic(features)
        
        action_probs = torch.softmax(action_logits, dim=-1)
        distribution = torch.distributions.Categorical(action_probs)
        
        actions = None
        if deterministic:
            actions = torch.argmax(action_probs, dim=-1)
        else:
            actions = distribution.sample()

        log_prob = distribution.log_prob(actions)
        
        return actions, value, log_prob

In [22]:
env = gym.make("procgen:procgen-coinrun-v0")

In [23]:
model = PPO(
        CnnPolicy,
        env,
        n_steps = 100, # timesteps before updating weights
        verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 100`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 36
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=100 and n_envs=1)
  warnings.warn(


In [24]:
model.learn(total_timesteps=1000)

/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-pack

----------------------------
| time/              |     |
|    fps             | 443 |
|    iterations      | 1   |
|    time_elapsed    | 0   |
|    total_timesteps | 100 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 339         |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 200         |
| train/                  |             |
|    approx_kl            | 0.020909222 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.7        |
|    explained_variance   | -0.0141     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.215       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0396     |
|    value_loss           | 2.29        |
-----------------------------------------
-----------------------------------------